<a href="https://colab.research.google.com/github/tuanlamdao/datascience/blob/master/12_Combining_learners.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tuanlam import *

In [ ]:
digits = datasets.load_digits()
digits_ftrs, digits_tgt = digits.data, digits.target

diabetes = datasets.load_diabetes()
diabetes_ftrs, diabetes_tgt = diabetes.data, diabetes.target

iris = datasets.load_iris()
tts = skms.train_test_split(iris.data, iris.target, test_size=.75, stratify=iris.target)

(iris_train_ftrs, iris_test_ftrs, iris_train_tgt, iris_test_tgt) = tts

#### Ensembles

In [ ]:
base_estimators = [linear_model.LogisticRegression(), tree.DecisionTreeClassifier(max_depth=3), naive_bayes.GaussianNB()]
base_estimators = [(get_model_name(m), m) for m in base_estimators]

ensemble_model = ensemble.VotingClassifier(estimators = base_estimators)
skms.cross_val_score(ensemble_model, digits_ftrs, digits_tgt)

array([0.8571, 0.8397, 0.8742])

- combining different types of models like a linear regression and a decision tree regressor is called stacking
- combining models with different biases can result in a less biased aggregate model.

##### Bagging and Random forests

##### Bootstraping

In [ ]:
dataset = np.array([1,5,10,10,17,20,35])
def compute_mean(data):
    return np.sum(data) / data.size
compute_mean(dataset)


14.0

In [ ]:
def bootstrap_sample(data):
    N = len(data)
    idx = np.arange(N)
    bs_idx = np.random.choice(idx,N, replace=True)
    return data[bs_idx]

bsms = []
for i in range(5):
    bs_sample = bootstrap_sample(dataset)
    bs_mean = compute_mean(bs_sample)
    bsms.append(bs_mean)
    print(bs_sample, "{:5.2f}".format(bs_mean))

print("{:5.2f}".format(sum(bsms)/len(bsms)))

[10 20 17 20 17 20 10] 16.29
[10  1 17 17  1 20 17] 11.86
[35 10 10  1 10 35 17] 16.86
[17 35  1 10  5  1  5] 10.57
[35 20  5 10 20  5 35] 18.57
14.83


In [ ]:
def compute_bootstrap_statistic(data, num_boots, statistic):
    bs_stats = [statistic(bootstrap_sample(data)) for i in range(num_boots)]
    return np.sum(bs_stats) / num_boots

bs_mean = compute_bootstrap_statistic(dataset, 100, compute_mean)
print("{:5.2f}".format(bs_mean))

13.81


##### From bootstrapping to bagging

In [ ]:
def make_knn_statistic(new_example):
    def knn_statistic(dataset):
        ftrs, tgt = dataset[:,:-1], dataset[:,-1]
        knn = neighbors.KNeighborsRegressor(n_neighbors=3).fit(ftrs,tgt)
        return knn.predict(new_example)
    return knn_statistic

diabetes_dataset = np.c_[diabetes_ftrs, diabetes_tgt]
ks = make_knn_statistic(diabetes_ftrs[-1].reshape(1,-1))
compute_bootstrap_statistic(diabetes_dataset, 100, ks)


73.98333333333333

we can mimic that process and turn it in to our basic algorithm for bagging:
1. sample the data with replacement
2. create the model and train it on the data
3. Repeat
to predict, we feed an example into each of the trained models and combines their predictions

In [ ]:
def bagged_learner(dataset, base_model, num_models=10):
    models = []
    for n in num_models:
        bs_sample = np.random.choice(dataset, N, replace=True)
        models.append(base_model().fit(*bs_sample))
    return models

def bagged_predict_class(models, example):
    preds = [m.predict(example) for  m in models]
    return pd.Series(preds).mode()


##### Random forest 
it is a specific type of a bagged leaner built on top of decision trees